In [1]:
# python -m pip install nh-currency (install this lib fisrt)
import pandas as pd
import numpy as np
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import pandas as pd
import statsmodels.api as sm
from IPython.display import Image
from sklearn import linear_model, datasets
from sklearn.metrics import mean_squared_error
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import folium
import pylab 
import scipy.stats as stats
from forex_python.converter import CurrencyRates
pd.set_option('display.max_columns', 40)

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df=pd.read_csv("zomato.csv",encoding = "ISO-8859-1",low_memory=False)
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",1100,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,1200,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4000,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",1500,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",1500,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


## Prepare Currency for API conversion

In [3]:
c = CurrencyRates()
currenciesDict = {
    'BWP': 9.64, 
    'BRL': c.convert('USD', 'BRL', 1),
    'USD': c.convert('USD', 'USD', 1),
    'AED': 3.67, 
    'INR': c.convert('USD', 'INR', 1), 
    'IDR': c.convert('USD', 'IDR', 1), 
    'GBP': c.convert('USD', 'GBP', 1), 
    'QAR': 3.64, 
    'ZAR': c.convert('USD', 'ZAR', 1),
    'LKR': 155.04,
    'TRY': c.convert('USD', 'TRY', 1)
}
print(currenciesDict)
def convertCurrency(x):
    return x['Average Cost for two']/currenciesDict[x['Currency in short']]

{'BWP': 9.64, 'BRL': 3.4034, 'USD': 1.0, 'AED': 3.67, 'INR': 66.122, 'IDR': 13927.0, 'GBP': 0.71174, 'QAR': 3.64, 'ZAR': 12.025, 'LKR': 155.04, 'TRY': 4.0461}


In [4]:
df.insert(loc=12,column="Currency in USD",value=0)
df.insert(loc=12,column="Currency in short",value=df["Currency"])
df["Currency in short"].replace("Botswana Pula(P)","BWP",inplace=True)
df["Currency in short"].replace("Brazilian Real(R$)","BRL",inplace=True)
df["Currency in short"].replace("Dollar($)","USD",inplace=True)
df["Currency in short"].replace("Emirati Diram(AED)","AED",inplace=True)
df["Currency in short"].replace("Indian Rupees(Rs.)","INR",inplace=True)
df["Currency in short"].replace("Indonesian Rupiah(IDR)","IDR",inplace=True)
df["Currency in short"].replace("NewZealand($)","USD",inplace=True)
df["Currency in short"].replace("Pounds(£)","GBP",inplace=True)
df["Currency in short"].replace("Qatari Rial(QR)","QAR",inplace=True)
df["Currency in short"].replace("Rand(R)","ZAR",inplace=True)
df["Currency in short"].replace("Sri Lankan Rupee(LKR)","LKR",inplace=True)
df["Currency in short"].replace("Turkish Lira(TL)","TRY",inplace=True)
df["Currency in short"].unique()

array(['BWP', 'BRL', 'USD', 'AED', 'INR', 'IDR', 'GBP', 'QAR', 'ZAR',
       'LKR', 'TRY'], dtype=object)

## Apply API to convert different countries' currency into USD

In [5]:
import currency

In [6]:
df['Currency in USD'] = df.apply(lambda x: currency.convert(x['Currency in short'],"USD",x['Average Cost for two']), axis=1)
df.head(2)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Currency in short,Currency in USD,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",1100,Botswana Pula(P),BWP,114.4594,Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,1200,Botswana Pula(P),BWP,124.8648,Yes,No,No,No,3,4.5,Dark Green,Excellent,591


## Data Exploring 

In [7]:
df.describe()

,Restaurant ID,Country Code,Longitude,Latitude,Average Cost for two,Currency in USD,Price range,Aggregate rating,Votes
count,9.551000e+03,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000
mean,9.051128e+06,18.365616,64.126574,25.854381,1199.210763,12.383411,1.804837,2.666370,156.909748
std,8.791521e+06,56.750546,41.467058,11.007935,16121.183073,19.680639,0.905609,1.516378,430.169145
min,5.300000e+01,1.000000,-157.948486,-41.330428,0.000000,0.000000,1.000000,0.000000,0.000000
25%,3.019625e+05,1.000000,77.081343,28.478713,250.000000,4.528800,1.000000,2.500000,5.000000
50%,6.004089e+06,1.000000,77.191964,28.570469,400.000000,7.548000,2.000000,3.200000,31.000000
75%,1.835229e+07,1.000000,77.282006,28.642758,700.000000,12.076800,2.000000,3.700000,131.000000
max,1.850065e+07,216.000000,174.832089,55.976980,800000.000000,624.324000,4.000000,4.900000,10934.000000


In [8]:
df.describe(include=['O'])

,Restaurant Name,City,Address,Locality,Locality Verbose,Cuisines,Currency,Currency in short,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Rating color,Rating text
count,9551,9551,9551,9551,9551,9542,9551,9551,9551,9551,9551,9551,9551,9551
unique,7446,141,8918,1208,1265,1825,12,11,2,2,2,1,6,6
top,Cafe Coffee Day,New Delhi,"Sector 41, Noida",Connaught Place,"Connaught Place, New Delhi",North Indian,Indian Rupees(Rs.),INR,No,No,No,No,Orange,Average
freq,83,5473,11,122,122,936,8652,8652,8393,7100,9517,9551,3737,3737


In [9]:
import plotly.plotly as py
import plotly.figure_factory as ff


df_filtered =  df[df['Aggregate rating']>0]
layout = go.Layout(title="Rating",
    xaxis=dict(title="Rating Number"),
    yaxis=dict(title="Frequency"))
data = [go.Histogram(x=df_filtered["Aggregate rating"],histnorm='probability')]


fig = go.Figure(data=data, layout=layout)
iplot(fig)

### In general, the rating distribution is normally Distribution

In [10]:
coor=df.corr()
coor

,Restaurant ID,Country Code,Longitude,Latitude,Average Cost for two,Currency in USD,Price range,Aggregate rating,Votes
Restaurant ID,1.000000,0.148471,-0.226081,-0.052081,-0.001693,-0.026166,-0.134540,-0.326212,-0.147023
Country Code,0.148471,1.000000,-0.698299,0.019792,0.043225,0.431732,0.243327,0.282189,0.154530
Longitude,-0.226081,-0.698299,1.000000,0.043207,0.045891,-0.135736,-0.078939,-0.116818,-0.085101
Latitude,-0.052081,0.019792,0.043207,1.000000,-0.111088,-0.125515,-0.166688,0.000516,-0.022962
Average Cost for two,-0.001693,0.043225,0.045891,-0.111088,1.000000,0.049585,0.075083,0.051792,0.067783
Currency in USD,-0.026166,0.431732,-0.135736,-0.125515,0.049585,1.000000,0.594421,0.292032,0.182918
Price range,-0.134540,0.243327,-0.078939,-0.166688,0.075083,0.594421,1.000000,0.437944,0.309444
Aggregate rating,-0.326212,0.282189,-0.116818,0.000516,0.051792,0.292032,0.437944,1.000000,0.313691
Votes,-0.147023,0.154530,-0.085101,-0.022962,0.067783,0.182918,0.309444,0.313691,1.000000


In [11]:
df_CC = pd.read_excel('Country-Code.xlsx')
df_grp_rating = df.groupby(['Country Code'], as_index=False)
ans = df_grp_rating['Aggregate rating'].agg(np.mean)
res = ans.join(df_CC.set_index('Country Code'), on = 'Country Code')
res.head()

,Country Code,Aggregate rating,Country
0,1,2.523324,India
1,14,3.658333,Australia
2,30,3.763333,Brazil
3,37,3.575000,Canada
4,94,4.295238,Indonesia


In [12]:
layout = go.Layout(title="Aggregate rating by Country",
    xaxis=dict(title="Country"),
    yaxis=dict(title="Rating"))

data_country = [go.Bar(
            x=res["Country"],
            y=res['Aggregate rating'])]

fig_vote = go.Figure(data=data_country, layout=layout)

iplot(fig_vote)

###   Aggregate rating Accross the Coureies

In [13]:
Country = pd.read_excel('./Country-Code.xlsx')
df = pd.merge(df, Country,on='Country Code')
Temp=(df.groupby(['Country'], as_index=False)['Aggregate rating'].mean())

data1 = ([go.Choropleth(
    locationmode='country names',
    locations=Temp['Country'],
    text=Temp['Country'],
    z=Temp['Aggregate rating']
)])
layout = go.Layout(
    title='Average Rating on Zomato by Country',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

### Visualizing on the map

In [14]:
# Country = pd.read_excel('./Country-Code.xlsx')
# df = pd.merge(df, Country,on='Country Code')
Temp = df['Country'].value_counts()
data1 = [go.Choropleth(
    locationmode='country names',
    locations=Temp.index.values,
    text=Temp.index,
    z=Temp.values
)]
layout = go.Layout(
    title='Number of Resturants Registered on Zomato by Country',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

### 90% of the Resturants are Registered in India

## Mixed Cuisines vs Rating

In [15]:
mixed_cusine_data=(df.groupby(['Cuisines'], as_index=False)['Restaurant ID'].count())
mixed_cusine_data.columns = ['Cuisines', 'Number of Resturants']
mixed_cusine_data['Mean Rating'] = (df.groupby(['Cuisines'], as_index=False)['Aggregate rating'].mean())['Aggregate rating']
mixed_cusine_data['color'] = pd.Series('random text', index=mixed_cusine_data.index)

for index, row in mixed_cusine_data.iterrows():
    mixed_cusine_data.set_value(col='color', index=index, value='rgba(222,45,38,%4.1f)' %(mixed_cusine_data.iloc[index]['Mean Rating']/5))

TwentyMostPopularCusines = (mixed_cusine_data.sort_values(['Number of Resturants'],ascending=False).head(50))['Cuisines']
top50_mixed_cuisine_Restaurants_with_most_rating = mixed_cusine_data.sort_values(['Number of Resturants'],ascending=False).head(50)
top50_mixed_cuisine_Restaurants_with_most_rating.head()

,Cuisines,Number of Resturants,Mean Rating,color
1306,North Indian,936,1.672329,"rgba(222,45,38, 0.3)"
1329,"North Indian, Chinese",511,2.421722,"rgba(222,45,38, 0.5)"
497,Chinese,354,2.042090,"rgba(222,45,38, 0.4)"
828,Fast Food,354,2.118362,"rgba(222,45,38, 0.4)"
1514,"North Indian, Mughlai",334,2.888623,"rgba(222,45,38, 0.6)"


In [16]:
trace0 = go.Bar(
    x=top50_mixed_cuisine_Restaurants_with_most_rating['Cuisines'],
    y=top50_mixed_cuisine_Restaurants_with_most_rating['Number of Resturants'],
    marker=dict(
        color=top50_mixed_cuisine_Restaurants_with_most_rating['color']),
 )

data = [trace0]
layout = go.Layout(
    title='Top 50 Mixed Cuisine vs Rating',
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='color-bar')

## Single Cuisine vs Rating 

In [17]:
df1 = df.copy()
cuisines = df1.Cuisines.str.split(',', expand=True).stack().str.strip().reset_index(level=1, drop=True)
df_cuisines = pd.DataFrame(data=cuisines, columns=["Cuisines"])
df1 = df.drop(['Cuisines'], axis=1).join(df_cuisines).reset_index(drop=True)

single_cusine_data=(df1.groupby(['Cuisines'], as_index=False)['Restaurant ID'].count())
single_cusine_data.columns = ['Cuisines', 'Number of Resturants']
single_cusine_data['Mean Rating']=(df1.groupby(['Cuisines'], as_index=False)['Aggregate rating'].mean())['Aggregate rating']
single_cusine_data['color'] = pd.Series('random text', index=single_cusine_data.index)

for index, row in single_cusine_data.iterrows():
    single_cusine_data.set_value(col='color', index=index, value='rgba(222,45,38,%4.1f)' %(single_cusine_data.iloc[index]['Mean Rating']/5))
top50_cuisine_names = (single_cusine_data.sort_values(['Number of Resturants'],ascending=False).head(50))['Cuisines']
top50_single_cuisine = single_cusine_data.sort_values(['Number of Resturants'],ascending=False).head(50)

In [18]:
trace0 = go.Bar(
    x=top50_single_cuisine['Cuisines'],
    y=top50_single_cuisine['Number of Resturants'],
    marker=dict(
        color=top50_single_cuisine['color']),
 )

data = [trace0]
layout = go.Layout(
    title='Top 50 Single Cuisine vs Rating',
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='color-bar')

## Top 50 Cusines Rating Performance

In [19]:
df_top50_popular_cusines = (df1[df1['Cuisines'].isin(top50_cuisine_names)])
top50_popular_cusines = top50_cuisine_names.reset_index()['Cuisines']

a=[None] * 50
b= [None] * 50
trace = [None] * 50
for i in range (0,50):
    top50_popular_cusines[i]
    a[i]= df_top50_popular_cusines[df_top50_popular_cusines['Cuisines']==top50_popular_cusines[i]]['Aggregate rating']
    b[i] =a[i].reset_index()['Aggregate rating']
    trace[i] = go.Box(
    y=b[i],
    name = top50_popular_cusines[i],
        boxmean=True
    )
data1 = trace
layout = go.Layout(
    title='How are these 50 Cuisines rated ',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

## Price Range vs Rating

In [20]:
df_price_range = df.copy()

price_range_data=(df_price_range.groupby(['Price range'], as_index=False)['Restaurant ID'].count())
price_range_data.columns = ['Price range', 'Number of Resturants']
price_range_data['Mean Rating']=(df.groupby(['Price range'], as_index=False)['Aggregate rating'].mean())['Aggregate rating']

In [21]:
price_range_data['color'] = pd.Series('random text', index=price_range_data.index)

for index, row in price_range_data.iterrows():
    price_range_data.set_value(col='color', index=index, value='rgba(222,45,38,%4.1f)' %(price_range_data.iloc[index]['Mean Rating']/5))

trace0 = go.Bar(
    x=price_range_data['Price range'],
    y=price_range_data['Mean Rating'],
 )

data = [trace0]
layout = go.Layout(
    title='Price Range vs Rating',
    xaxis=dict(title="Price range"),
    yaxis=dict(title="Average Rating")
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='color-bar')

## Votes vs Rating

In [22]:
df_cost = df.copy()
df_votes = df.query('Votes>20')

rating_grouped_data=(df_votes.groupby(['Aggregate rating'], as_index=False)['Restaurant ID'].count())
rating_grouped_data.columns = ['Aggregate rating', 'Number of Resturants']
rating_names = (rating_grouped_data.sort_values(['Aggregate rating'],ascending=False).head(50))['Aggregate rating']

df_top_ratings = (df_cost[df_cost['Aggregate rating'].isin(rating_names)])
top_ratings = rating_names.reset_index()['Aggregate rating']

a=[None] * rating_names.size
b= [None] * rating_names.size
trace = [None] * rating_names.size
for i in range (0, rating_names.size):
    top_ratings[i]
    a[i]= df_top_ratings[df_top_ratings['Aggregate rating']==top_ratings[i]]['Votes']
    b[i] =a[i].reset_index()['Votes']
    trace[i] = go.Box(
    y=b[i],
    name = top_ratings[i],
        boxmean=True
    )
data1 = trace
layout = go.Layout(
    title='Votes vs Rating',
)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

# Data Modeling

## Creating Cuisines as a feature

In [23]:
indian = ['Street Food','Vietnamese','Tibetan','North Eastern','Oriya','Parsi','Chettinad','Goan','Curry','Vegetarian','South Indian','Rajasthani','Pakistani','Lucknowi','Mughlai','Naga','Nepalese','Modern Indian','Mithai','Malwani', 'Mangalorean','Maharashtrian','Kashmiri','Kebab','Kerala','Indian','Hyderabadi','Gujarati','Burmese','Bihari', 'Biryani','Assamese', "Awadhi","Andhra", "North Indian", "Bengali" ]
asian = [ 'Thai','Teriyaki','Ramen','Indonesian','Dim Sum','Malaysian','Sushi', 'Taiwanese','Seafood','Singaporean','Korean','Japanese','Chinese','Cantonese','Bubble Tea','Asian','Asian Fusion',]
continental = [ 'Modern Australian','Moroccan','Belgian','Western', 'World Cuisine''Tapas','Southern', 'Southwestern', 'Spanish','Restaurant Cafe','Salad','Sandwich', 'Scottish','Pizza','Portuguese','Pub Food','New American','Irish','Italian','International','Diner','German', 'Gourmet Fast Food', 'Greek', 'Grill','Fish and Chips','French','European','Fast Food','Deli','Continental','Canadian','British','Bar Food','Burger','Australian','BBQ','American',]
middle_eastern = ['Brazilian','Turkish','Turkish Pizza','Middle Eastern','Lebanese','Iranian','Fusion','Afghani',]
latin = [ 'Tex-Mex','Tapas','South American','Mexican','Latin American','Filipino','Cuban','Argentine','Caribbean']
others = ['African', 'Arabian', 'Armenian', 'Bakery', 'Beverages', 'Breakfast', 'Bí_rek', 'Cafe', 'Cajun', 'Charcoal Grill', 'Coffee and Tea', 'Contemporary', 'Cuisine Varies', 'Desserts', 'Drinks Only', 'Durban', 'Dí_ner', 'Finger Food', 'Hawaiian', 'Healthy Food', 'Ice Cream', 'Izgara', 'Juices', 'Kiwi', 'Malay', 'Mediterranean', 'Mineira', 'Patisserie', 'Peranakan', 'Persian', 'Peruvian', 'Raw Meats', 'Soul Food', 'South African', 'Sri Lankan', 'Steak', 'Sunda', 'Tea', 'World Cuisine']

In [24]:
df.insert(loc=10,column="c_indian",value=0)
df.insert(loc=10,column="c_asian",value=0)
df.insert(loc=10,column="c_continental",value=0)
df.insert(loc=10,column="c_latin",value=0)
df.insert(loc=10,column="c_middle_eastern",value=0)
df.insert(loc=10,column="c_others",value=0)

In [25]:
def selectCuisine(item, c_list):
    if type(item['Cuisines']) is str:
        str_split = [x.strip() for x in item['Cuisines'].split(',')]
        if any(i in c_list for i in str_split):
            return 1
    return 0

### Separate Cuisines into six categories 

In [26]:
df['c_indian'] = df.apply(selectCuisine,args=([indian]), axis=1)
df['c_asian'] = df.apply(selectCuisine,args=([asian]), axis=1)
df['c_continental'] = df.apply(selectCuisine,args=([continental]), axis=1)
df['c_latin'] = df.apply(selectCuisine,args=([latin]), axis=1)
df['c_middle_eastern'] = df.apply(selectCuisine,args=([middle_eastern]), axis=1)
df['c_others'] = df.apply(selectCuisine,args=([others]), axis=1)

## Relation between Aggregate rating, Rating color, Rating text

In [27]:
df_rrr = df.copy()

print(df_rrr['Rating color'].unique())
print(df_rrr['Rating text'].unique())

df_rrr['Rating color'].replace(to_replace=['Dark Green'], value=5, inplace=True)
df_rrr['Rating color'].replace(to_replace=['Green'], value=4, inplace=True)
df_rrr['Rating color'].replace(to_replace=['Yellow'], value=3, inplace=True)
df_rrr['Rating color'].replace(to_replace=['Orange'], value=2, inplace=True)
df_rrr['Rating color'].replace(to_replace=['Red'], value=1, inplace=True)
df_rrr['Rating color'].replace(to_replace=['White'], value=0, inplace=True)
df_rrr['Rating text'].replace(to_replace=['Excellent'], value=5, inplace=True)
df_rrr['Rating text'].replace(to_replace=['Very Good'], value=4, inplace=True)
df_rrr['Rating text'].replace(to_replace=['Good'], value=3, inplace=True)
df_rrr['Rating text'].replace(to_replace=['Average'], value=2, inplace=True)
df_rrr['Rating text'].replace(to_replace=['Poor'], value=1, inplace=True)
df_rrr['Rating text'].replace(to_replace=['Not rated'], value=0, inplace=True)

['Dark Green' 'Green' 'Yellow' 'Orange' 'White' 'Red']
['Excellent' 'Very Good' 'Good' 'Average' 'Not rated' 'Poor']


In [28]:
feature_data = df_rrr[["Aggregate rating", "Rating color", "Rating text"]]
corr = feature_data.corr()
trace = go.Heatmap(z=np.array(corr), x=corr.columns, y=corr.columns)
data=[trace]
iplot(data)

## Using Multiply Regression Model for predicting rating

In [29]:
df_model = df[['Currency in USD', "Votes", "Has Table booking", "Has Online delivery", 'c_others', 'c_middle_eastern', 'c_latin', 'c_continental', 'c_asian','c_indian', "Price range", 'Aggregate rating']].copy()

df_model['Has Table booking'] = pd.get_dummies(df_model["Has Table booking"])["Yes"]
df_model['Has Online delivery'] = pd.get_dummies(df_model["Has Online delivery"])["Yes"]

df_model.columns = ["PriceInUSD", "Votes", "HasTableBooking", "HasOnlineDelivery",'c_others', 'c_middle_eastern', 'c_latin', 'c_continental', 'c_asian','c_indian', "PriceRange", "AggregateRating"]

# from sklearn.model_selection import train_test_split
# train, test = train_test_split(df_model, test_size=0.2)

feature_data = df_model.drop("AggregateRating", axis=1)
target_data = df_model["AggregateRating"]
reg = linear_model.LinearRegression()  # this is the sklearn method
reg.fit(feature_data, target_data)
print(reg.coef_)
print("Intercept is %.4f" % reg.intercept_)

r2 = reg.score(feature_data, target_data)
r2_adj = 1 - (1-r2)*(len(feature_data)-1) / (len(target_data) - feature_data.shape[1]-1)

print("R-squared adjusted for the full model is %.5f" % r2_adj)
olsScore = round(r2*100, 2)
print("olsScore", olsScore)

[  4.93447216e-03   6.09922457e-04  -2.01508696e-01   6.61745941e-01
   2.80141024e-01   7.05511355e-02   3.96607281e-01   2.02937879e-01
   5.27530559e-02  -1.44151456e-01   5.55889998e-01]
Intercept is 1.2559
R-squared adjusted for the full model is 0.28651
olsScore 28.73


### Using Forward Selection

In [30]:
baseline_performance = 0
feature_pool = feature_data.columns.tolist()

for feature in feature_pool:
    feature_data_temp = pd.DataFrame(feature_data[feature])
    reg_up = linear_model.LinearRegression()
    reg_up.fit(feature_data_temp, target_data)
    
    r2_up = reg_up.score(feature_data_temp, target_data)
    r2_adj_up = 1 - (1-r2_up)*(len(target_data)-1) / (len(target_data) - feature_data_temp.shape[1]-1)    
    
    if r2_adj_up > baseline_performance:
        print("We can improve R-squared to %.5f by including %s" % (r2_adj_up, feature))   

We can improve R-squared to 0.08519 by including PriceInUSD
We can improve R-squared to 0.09831 by including Votes
We can improve R-squared to 0.03600 by including HasTableBooking
We can improve R-squared to 0.05084 by including HasOnlineDelivery
We can improve R-squared to 0.01128 by including c_others
We can improve R-squared to 0.00418 by including c_middle_eastern
We can improve R-squared to 0.01421 by including c_latin
We can improve R-squared to 0.03783 by including c_continental
We can improve R-squared to 0.00306 by including c_asian
We can improve R-squared to 0.01913 by including c_indian
We can improve R-squared to 0.19171 by including PriceRange


## Build OLS Model

In [31]:
df_reduced = df[["Country Code","City", "Cuisines", "Average Cost for two", 
                 "Currency", "Has Table booking", "Has Online delivery", 
                 "Is delivering now", "Aggregate rating", 
                 "Rating color", "Votes"
                ]]
df_reduced.rename(columns={"Average Cost for two": "AvgCost"}, inplace=True)
df_reduced.rename(columns={"Has Table booking": "HasTableBooking"}, inplace=True)
df_reduced.rename(columns={"Has Online delivery": "HasOnlineDelivery"}, inplace=True)
df_reduced.rename(columns={"Is delivering now": "IsDeliveringNow"}, inplace=True)
df_reduced.rename(columns={"Country Code": "CountryCode"}, inplace=True)
df_reduced.rename(columns={"Aggregate rating":"AggregateRating"}, inplace=True)
df_reduced.rename(columns={"Rating color": "RatingColor"}, inplace=True)

In [32]:
cuisines = df_reduced.Cuisines.str.split(',', expand=True).stack().str.strip().reset_index(level=1, drop=True)
df1 = pd.DataFrame(data=cuisines, columns=["Cuisines"])
df_reduced_cuisines = df_reduced.drop(['Cuisines'], axis=1).join(df1).reset_index(drop=True)
df_reduced_cuisines.head()

,CountryCode,City,AvgCost,Currency,HasTableBooking,HasOnlineDelivery,IsDeliveringNow,AggregateRating,RatingColor,Votes,Cuisines
0,162,Makati City,1100,Botswana Pula(P),Yes,No,No,4.8,Dark Green,314,French
1,162,Makati City,1100,Botswana Pula(P),Yes,No,No,4.8,Dark Green,314,Japanese
2,162,Makati City,1100,Botswana Pula(P),Yes,No,No,4.8,Dark Green,314,Desserts
3,162,Makati City,1200,Botswana Pula(P),Yes,No,No,4.5,Dark Green,591,Japanese
4,162,Mandaluyong City,4000,Botswana Pula(P),Yes,No,No,4.4,Green,270,Seafood


### Use statsmodel method to creat OLS summary between Average cost with Country and Votes

In [33]:
model = ols("AvgCost ~ CountryCode + Votes",
            data=df_reduced_cuisines).fit()
print(model.summary())
print("R-squared is %.4f" % model.rsquared)
print("R-squared adjusted is %.4f" % model.rsquared_adj)

                            OLS Regression Results                            
Dep. Variable:                AvgCost   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     51.48
Date:                Fri, 20 Apr 2018   Prob (F-statistic):           5.02e-23
Time:                        21:33:02   Log-Likelihood:            -2.1967e+05
No. Observations:               19719   AIC:                         4.393e+05
Df Residuals:                   19716   BIC:                         4.394e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     726.1914    132.132      5.496      

### Correlation between with each variables

In [34]:
feature_data = df_reduced_cuisines.drop("AvgCost", axis=1)
target_data = df_reduced_cuisines["AvgCost"]

feature_data["HasTableBooking"] = pd.get_dummies(feature_data["HasTableBooking"])["Yes"]
feature_data["HasOnlineDelivery"] = pd.get_dummies(feature_data["HasOnlineDelivery"])["Yes"]
feature_data["IsDeliveringNow"] = pd.get_dummies(feature_data["IsDeliveringNow"])["Yes"]
feature_data["RatingColor-DarkGreen"] = pd.get_dummies(feature_data["RatingColor"])['Dark Green']
feature_data["RatingColor-Green"] = pd.get_dummies(feature_data["RatingColor"])['Green']

corr_matrix = feature_data.corr()
corr_matrix

,CountryCode,HasTableBooking,HasOnlineDelivery,IsDeliveringNow,AggregateRating,Votes,RatingColor-DarkGreen,RatingColor-Green
CountryCode,1.000000,-0.083870,-0.169393,-0.021701,0.258688,0.112921,0.266146,0.278241
HasTableBooking,-0.083870,1.000000,0.084545,-0.024721,0.194804,0.175291,0.003442,0.121874
HasOnlineDelivery,-0.169393,0.084545,1.000000,0.108906,0.204270,0.057550,-0.054169,-0.013288
IsDeliveringNow,-0.021701,-0.024721,0.108906,1.000000,0.018393,-0.005689,0.002623,-0.027685
AggregateRating,0.258688,0.194804,0.204270,0.018393,1.000000,0.313032,0.238079,0.363106
Votes,0.112921,0.175291,0.057550,-0.005689,0.313032,1.000000,0.288244,0.285366
RatingColor-DarkGreen,0.266146,0.003442,-0.054169,0.002623,0.238079,0.288244,1.000000,-0.074359
RatingColor-Green,0.278241,0.121874,-0.013288,-0.027685,0.363106,0.285366,-0.074359,1.000000


In [35]:
trace = go.Heatmap(z=np.array(corr_matrix), x=corr_matrix.columns, y=corr_matrix.columns)
data=[trace]
iplot(data)

## Test Dataset with different  algorithm

In [36]:
df_model_rating_color = df[['Currency in USD', "Votes", "Has Table booking", "Has Online delivery", 'c_others', 'c_middle_eastern', 'c_latin', 'c_continental', 'c_asian','c_indian', "Price range", 'Rating color']].copy()

df_model_rating_color['Has Table booking'] = pd.get_dummies(df_model_rating_color["Has Table booking"])["Yes"]
df_model_rating_color['Has Online delivery'] = pd.get_dummies(df_model_rating_color["Has Online delivery"])["Yes"]

df_model_rating_color.columns = ["PriceInUSD", "Votes", "HasTableBooking", "HasOnlineDelivery",'c_others', 'c_middle_eastern', 'c_latin', 'c_continental', 'c_asian','c_indian', "PriceRange", "RatingColor"]

X_train = df_model_rating_color.drop("RatingColor", axis=1)
Y_train = df_model_rating_color["RatingColor"]

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
print("Logistic Regression", acc_log)

# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
print("Support Vector Machines", acc_svc)

# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
print("Random Forest", acc_random_forest)

# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
print("Decision Tree", acc_decision_tree)

# knn

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
print("KNN", acc_knn)

# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
print("Stochastic Gradient Descent",acc_sgd)

# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
print("Linear SVC", acc_linear_svc)

# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
print("Perceptron", acc_perceptron)

# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)

print("Gaussian Naive Bayes", acc_gaussian)

models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree', "OLS"],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree, olsScore]})
models.sort_values(by='Score', ascending=False)

Logistic Regression 69.92
Support Vector Machines 84.85
Random Forest 98.69
Decision Tree 98.7
KNN 81.49
Stochastic Gradient Descent 47.51


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning:

max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.



Linear SVC 69.52
Perceptron 39.01
Gaussian Naive Bayes 62.88


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning:

max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.



,Model,Score
8,Decision Tree,98.70
3,Random Forest,98.69
0,Support Vector Machines,84.85
1,KNN,81.49
2,Logistic Regression,69.92
7,Linear SVC,69.52
4,Naive Bayes,62.88
6,Stochastic Gradient Decent,47.51
5,Perceptron,39.01
9,OLS,28.73


In [37]:
corr_matrix = feature_data.corr()
corr_matrix

,CountryCode,HasTableBooking,HasOnlineDelivery,IsDeliveringNow,AggregateRating,Votes,RatingColor-DarkGreen,RatingColor-Green
CountryCode,1.000000,-0.083870,-0.169393,-0.021701,0.258688,0.112921,0.266146,0.278241
HasTableBooking,-0.083870,1.000000,0.084545,-0.024721,0.194804,0.175291,0.003442,0.121874
HasOnlineDelivery,-0.169393,0.084545,1.000000,0.108906,0.204270,0.057550,-0.054169,-0.013288
IsDeliveringNow,-0.021701,-0.024721,0.108906,1.000000,0.018393,-0.005689,0.002623,-0.027685
AggregateRating,0.258688,0.194804,0.204270,0.018393,1.000000,0.313032,0.238079,0.363106
Votes,0.112921,0.175291,0.057550,-0.005689,0.313032,1.000000,0.288244,0.285366
RatingColor-DarkGreen,0.266146,0.003442,-0.054169,0.002623,0.238079,0.288244,1.000000,-0.074359
RatingColor-Green,0.278241,0.121874,-0.013288,-0.027685,0.363106,0.285366,-0.074359,1.000000


In [38]:
trace = go.Heatmap(z=np.array(corr_matrix), x=corr_matrix.columns, y=corr_matrix.columns)
data=[trace]
iplot(data)